## 5. Build Iot Edge Module - Docker Image
If not already, you should run first jupyter notebook (01_setup_environment.ipynb) in this sample to set the global variables.

### 5.1. Get global variables
We will read the previously stored variables

In [ ]:
from dotenv import set_key, get_key, find_dotenv
envPath = find_dotenv(raise_error_if_not_found=True)

isSolutionPath = get_key(envPath, "isSolutionPath")
containerImageName = get_key(envPath, "containerImageName")
containerImageName = get_key(envPath, "containerImageName")
localacrServiceName = get_key(envPath, "localacrServiceName")
acrServiceName = get_key(envPath, "acrServiceName")

In [ ]:
# remove possible python cache from previous local tests
!rm -rf $isSolutionPath/__pycache__

### 5.2 Create Web Application & Server for our ml solution

In [ ]:
%%writefile $isSolutionPath/app.py

import os
from flask import Flask, request, Response
import cv2
import logging
import json
import io
from score import AnalyticsAPI
import numpy as np
import time
import datetime

import threading
from azure.iot.device import IoTHubModuleClient, MethodResponse

# Initial settings of AI model
AIX_MODEL_NAME = "person-vehicle-bike-detection-crossroad-1016" # see AnalyticsAPI class for full list of models and other possibilities
AIX_TARGET_DEVICE = "CPU"
AIX_MODEL_PRECISION = "FP32"
AIX_PROBABILITY_THRESHOLD = 0.5
AIX_DEBUG_PATH = "/debug_path/" # Same value used in Manifest file. So changing this value will affect the manifest file's consistancy

app = Flask(__name__)
analyticsAPI = AnalyticsAPI(    modelName=AIX_MODEL_NAME, 
                                modelPrecision=AIX_MODEL_PRECISION, 
                                targetDev=AIX_TARGET_DEVICE, 
                                probThreshold=AIX_PROBABILITY_THRESHOLD)

# Wait some time to have the API - HW accelerators initialized
time.sleep(2)
logger = analyticsAPI.logger

# Debug flag for writing out processed frames with bounding boxes etc. marked on them.
# if set, debug data will be written in to "/debug_path" folder, which is bind to host PC's local drive.
# Host PC local drive is specified in the Manifest file, and default value is host PC's /tmp/aix_debug folder.
# /tmp already exist in host pc but you should manually create and give access credentails to /tmp/aix_debug (or any directory you want...)
#
# 0 : no debug output
# 1 : output frames only if object detected
# 2 : output all frames received
#
AIX_DEBUG = 2

# IoT Hub cloud to device method handles
# https://docs.microsoft.com/en-us/python/api/azure-iot-device/azure.iot.device?view=azure-python (see IoTHubModuleClient)
#
def directMethodListener(client):
    global analyticsAPI
    global AIX_DEBUG
    while True:
        try:
            methodRequest = client.receive_method_request()  # blocking call
            logger.info("[AI EXT] Method invoked: {0} - {1}".format(methodRequest.name, methodRequest.payload))

            if methodRequest.name == "SetProbabilityThreshold":
                analyticsAPI.setProbabilityThreshold(float(methodRequest.payload))
                callResult = str(analyticsAPI.getProbabilityThreshold())

            elif methodRequest.name == "SetModel":
                analyticsAPI.setModelName(str(methodRequest.payload))
                callResult = analyticsAPI.getModelName()

            elif methodRequest.name == "SetModelPrecision":
                analyticsAPI.setModelPrecision(str(methodRequest.payload))
                callResult = analyticsAPI.getModelPrecision()

            elif methodRequest.name == "SetTargetDevice":
                analyticsAPI.setTargetDevice(str(methodRequest.payload))
                callResult = analyticsAPI.getTargetDevice()

            elif methodRequest.name == "SetDebug":
                AIX_DEBUG = int(methodRequest.payload)
                callResult = str(AIX_DEBUG)

            else:
                continue

            responsePayload = {"Success": "{0} - {1}".format(methodRequest.name, callResult)}
            methodResponse = MethodResponse(methodRequest.request_id, 200, payload=responsePayload)
            client.send_method_response(methodResponse)

        except Exception as e:
            responsePayload = {"Exception": "{0} - {1}".format(methodRequest.name, str(e))}
            methodResponse = MethodResponse(methodRequest.request_id, 400, payload=responsePayload)
            client.send_method_response(methodResponse)

#
# Initialize IoTHubDeviceClient to listen method calls coming from cloud.
#
iotHubModuleClient = IoTHubModuleClient.create_from_edge_environment(websockets=True)
iotHubModuleClient.connect()

# Start IoTHub event callback threads
threadDirectMethodListener = threading.Thread(target=directMethodListener, args=(iotHubModuleClient,))
threadDirectMethodListener.daemon = True
threadDirectMethodListener.start()

# TODO: handle twin updates...

@app.route("/score", methods = ['POST'])
def scoreRRS():
    global analyticsAPI
    global logger
    global AIX_DEBUG
    
    # uncomment for extended debug log
    # logger.info("[AI EXT] Received scoring request. Header: {0}".format(json.dumps(dict(request.headers))))
    # logger.info("[AI EXT] Received scoring request")

    try:
        # We accept only "image/jpeg" but there is no reason to not to update below line and allow PNG etc. formats as OpenCV supports many of them...
        if request.headers['Content-Type'] != 'image/jpeg':
            logger.info("[AI EXT] Non JPEG content sent. Exiting the scoring event...")
            return Response(json.dumps({}), status= 415, mimetype ='application/json')

        # Before going forward, return not ready, if it is the case...
        if not analyticsAPI.initialized:
            return  Response(json.dumps({"status": analyticsAPI.INF_STAT_NOT_READY}), status= 200, mimetype ='application/json')  

        # get request as byte stream
        reqBody = request.get_data(False)

        # convert from byte stream
        inMemFile = io.BytesIO(reqBody)

        # load a sample image
        inMemFile.seek(0)
        fileBytes = np.asarray(bytearray(inMemFile.read()), dtype=np.uint8)
        cvImage = cv2.imdecode(fileBytes, cv2.IMREAD_COLOR)

        # call scoring function
        result = analyticsAPI.score(cvImage)            

        color = (255, 0, 0) # in BGR order

        if AIX_DEBUG > 0:
            jsonResult = json.loads(result)
            statusCode = int(jsonResult["status"])
            objectCount = 0
            
            if statusCode == 0:
                objectCount = int(jsonResult["object_count"])
                if objectCount > 0:
                    jsonResult = jsonResult["result"]
                    for k in jsonResult:
                        detRes = jsonResult[k]
                        xmin = int(detRes["xmin"])
                        ymin = int(detRes["ymin"])
                        xmax = int(detRes["xmax"])
                        ymax = int(detRes["ymax"])
                        classId = str(detRes["label"])
                        confidence = str(detRes["confidence"])

                        cv2.rectangle(cvImage, (xmin, ymin), (xmax, ymax), color, 2)
                        cv2.putText(cvImage, classId + " - " + confidence, (xmin, ymin - 7), cv2.FONT_HERSHEY_COMPLEX, 1, color, 1)

            if (AIX_DEBUG > 1) or (objectCount > 0):
                camID = request.args.get('cam', default = 0, type = int)
                outPath = os.path.join(AIX_DEBUG_PATH, "cam{0:02d}".format(camID))
                os.makedirs(outPath, exist_ok=True)
                outFileName = os.path.join(outPath, str(datetime.datetime.now()).replace(" ","_") + ".jpg")

                # resizedImg = cv2.resize(cvImage, (800, 600), interpolation = cv2.INTER_AREA)
                # retval = cv2.imwrite(outFileName, resizedImg)
                retval = cv2.imwrite(outFileName, cvImage)

            # Log only in debug mode
            logger.info("[AI EXT] Sending response. Status: {0} - Count: {1}".format(statusCode, objectCount))

        logger.info("[AI EXT] Responding scoring request.")
        return Response(result, status= 200, mimetype ='application/json')

    except Exception as e:
        logger.info("[AI EXT] Exception (scoreRRS): {0}".format(str(e)))
        return Response(json.dumps({}), status= 200, mimetype ='application/json')   
    
@app.route("/")
def healthy():
    return "Healthy"

# Version
@app.route('/version', methods = ['GET'])
def version_request():
    global analyticsAPI
    return analyticsAPI.version()

# About
@app.route('/about', methods = ['GET'])
def about_request():
    global analyticsAPI
    return analyticsAPI.about() + "<br><br> AIX_DEBUG: " + str(AIX_DEBUG)

if __name__ == "__main__":
    while not analyticsAPI.initialized:
        logger.info("[AI EXT] Waiting AI module to be initialized. (app.py)")
        time.sleep(1)
        
    app.run(host='0.0.0.0', port=5444)

No need to make any change here but 5444 is the internal port of the webserver app that listens the requests. Later we will map it to different port to expose to external (see next cell)

In [ ]:
%%writefile $isSolutionPath/wsgi.py
from app import app as application

def create():
    print("[AI EXT] Initialising lva ai extension web app")
    application.run(host='127.0.0.1', port=5444)

In [ ]:
import os
os.makedirs(os.path.join(isSolutionPath, "nginx"), exist_ok=True)

Exposed port of the web app is now 5001 (while the internal one is 5444)

In [ ]:
%%writefile $isSolutionPath/nginx/app
server {
    listen 5001;
    server_name _;
 
    location / {
    include proxy_params;
    proxy_pass http://127.0.0.1:5444;
    proxy_connect_timeout 5000s;
    proxy_read_timeout 5000s;
  }
}

In [ ]:
%%writefile $isSolutionPath/gunicorn_logging.conf

[loggers]
keys=root, gunicorn.error

[handlers]
keys=console

[formatters]
keys=json

[logger_root]
level=INFO
handlers=console

[logger_gunicorn.error]
level=ERROR
handlers=console
propagate=0
qualname=gunicorn.error

[handler_console]
class=StreamHandler
formatter=json
args=(sys.stdout, )

[formatter_json]
class=jsonlogging.JSONFormatter

In [ ]:
%%writefile $isSolutionPath/kill_supervisor.py
import sys
import os
import signal

def write_stdout(s):
    sys.stdout.write(s)
    sys.stdout.flush()

# this function is modified from the code and knowledge found here: http://supervisord.org/events.html#example-event-listener-implementation
def main():
    while 1:
        write_stdout('[AI EXT] READY\n')
        # wait for the event on stdin that supervisord will send
        line = sys.stdin.readline()
        write_stdout('[AI EXT] Killing supervisor with this event: ' + line);
        try:
            # supervisord writes its pid to its file from which we read it here, see supervisord.conf
            pidfile = open('/tmp/supervisord.pid','r')
            pid = int(pidfile.readline());
            os.kill(pid, signal.SIGQUIT)
        except Exception as e:
            write_stdout('[AI EXT] Could not kill supervisor: ' + e.strerror + '\n')
            write_stdout('[AI EXT] RESULT 2\nOK')

main()

In [ ]:
import os
os.makedirs(os.path.join(isSolutionPath, "etc"), exist_ok=True)

In [ ]:
%%writefile $isSolutionPath/etc/supervisord.conf 
[supervisord]
logfile=/tmp/supervisord.log ; (main log file;default $CWD/supervisord.log)
logfile_maxbytes=50MB        ; (max main logfile bytes b4 rotation;default 50MB)
logfile_backups=10           ; (num of main logfile rotation backups;default 10)
loglevel=info                ; (log level;default info; others: debug,warn,trace)
pidfile=/tmp/supervisord.pid ; (supervisord pidfile;default supervisord.pid)
nodaemon=true                ; (start in foreground if true;default false)
minfds=1024                  ; (min. avail startup file descriptors;default 1024)
minprocs=200                 ; (min. avail process descriptors;default 200)
environment=LD_LIBRARY_PATH=%(ENV_LD_LIBRARY_PATH)s,INTEL_CVSDK_DIR=%(ENV_INTEL_CVSDK_DIR)s,OpenCV_DIR=%(ENV_OpenCV_DIR)s,InferenceEngine_DIR=%(ENV_InferenceEngine_DIR)s,PYTHONPATH=%(ENV_PYTHONPATH)s,INTEL_OPENVINO_DIR=%(ENV_INTEL_OPENVINO_DIR)s,PATH=%(ENV_PATH)s,HDDL_INSTALL_DIR=%(ENV_HDDL_INSTALL_DIR)s,INTEL_OPENVINO_DIR=%(ENV_INTEL_OPENVINO_DIR)s,PATH=%(ENV_PATH)s

[program:gunicorn]
command=bash -c "gunicorn --workers 1 -m 007 --timeout 100000 --capture-output --error-logfile - --log-level debug --log-config gunicorn_logging.conf \"wsgi:create()\""
directory=/code
redirect_stderr=true
stdout_logfile =/dev/stdout
stdout_logfile_maxbytes=0
startretries=2
startsecs=20

[program:nginx]
command=/usr/sbin/nginx -g "daemon off;"
startretries=2
startsecs=5
priority=3

[eventlistener:program_exit]
command=python kill_supervisor.py
directory=/code
events=PROCESS_STATE_FATAL
priority=2

In [ ]:
%%writefile $isSolutionPath/requirements.txt
pillow<7.0.0
click==6.7
configparser==3.5.0
Flask==0.12.2
gunicorn==19.6.0
json-logging-py==0.2
MarkupSafe==1.0
olefile==0.44
requests==2.12.3

### 5.3. Create Docker File to containerize ml solution and web app server

TODO: Update Dockerfile to reduce its size... There exist many ways to do it but for simplicity and self explainability we keep as is below.

<span style="color:red; font-weight: bold; font-size:1.1em;"> [!Important] </span>  

OpenVino Toolkit is a licenced software. Below link probably will not be working for you and you should replace it with your own offline download link. To get your own download link, follow the below instructions:  

1) Goto address: https://software.intel.com/en-us/openvino-toolkit/choose-download/free-download-linux  
2) Click on "Register & Download" button  
<img src="../doc_imgs/img_03_001.jpg" width=400 alt="> Figure: Register & Download."/>  

3) Fill in the form and submit  
<img src="../doc_imgs/img_03_002.jpg" width=400 alt="> Figure: Fill the form."/>  

4) Over the "Full Package" link, right click and get the link which should look like something:  
    http://registrationcenter-download.intel.com/akdlm/irc_nas/<SOMECODE\>/l_openvino_toolkit_p_2020.1.023.tgz  
<img src="../doc_imgs/img_03_003.jpg" width=400 alt="> Figure: Download link."/>  

5) In the below cell, set the value of variable "openVinoToolkitDownloadLink" to the download link you have

In [ ]:
# As described above, set the value of variable "openVinoToolkitDownloadLink" to the download link you have (below is sample URI, just remove it and use your own address)
openVinoToolkitDownloadLink = "http://registrationcenter-download.intel.com/akdlm/irc_nas/16345/l_openvino_toolkit_p_2020.1.023.tgz"

In [ ]:
%%writefile $isSolutionPath/Dockerfile

FROM ubuntu:18.04
MAINTAINER Mustafa K. <mkasap@microsoft.com>

USER root

ARG WORK_DIR=/code
ENV WORK_DIR ${WORK_DIR}
ENV PATH /opt/miniconda/bin:${PATH}

RUN mkdir -p ${WORK_DIR}

WORKDIR ${WORK_DIR}

#
# Install base
#
RUN apt-get update &&\
    apt-get install -y --no-install-recommends \
        # Essentials
        wget \
        locales \
        # Python environment
        python3 \
        python3-setuptools &&\
    #
    # Dependencies: conda
    wget --quiet https://repo.anaconda.com/miniconda/Miniconda3-4.5.11-Linux-x86_64.sh -O ${WORK_DIR}/miniconda.sh --no-check-certificate &&\ 
    /bin/bash ${WORK_DIR}/miniconda.sh -b -p /opt/miniconda &&\
    #
    # Cleaning
    /opt/miniconda/bin/conda clean -ya &&\
    rm -rf /opt/miniconda/pkgs &&\
    rm -rf /var/lib/apt/lists/*

#
# Install OpenVino
#
#COPY l_openvino_toolkit_p_2020.1.023.tgz ${WORK_DIR}
RUN apt-get update &&\
    apt-get install -y --no-install-recommends \
        # Essentials
        cpio \
        udev \
        unzip \
        autoconf \
        automake \
        libtool

RUN wget --quiet AIX_OPENVINO_TOOLKIT_DOWNLOAD_LINK -O ${WORK_DIR}/l_openvino_toolkit_p_2020.1.023.tgz &&\
    pattern="COMPONENTS=DEFAULTS" &&\
    replacement="COMPONENTS=intel-openvino-ie-sdk-ubuntu-bionic__x86_64;intel-openvino-ie-rt-cpu-ubuntu-bionic__x86_64;intel-openvino-ie-rt-vpu-ubuntu-bionic__x86_64;intel-openvino-opencv-lib-ubuntu-bionic__x86_64" &&\
    tar -xzf l_openvino_toolkit*.tgz &&\
    cd l_openvino_toolkit* &&\
    sed -i "s/$pattern/$replacement/" silent.cfg &&\
    sed -i "s/decline/accept/g" silent.cfg &&\
    /bin/bash ./install.sh -s silent.cfg &&\
    cd - &&\
    cd /opt/intel/openvino/install_dependencies &&\
    /bin/bash ./install_openvino_dependencies.sh &&\
    # setup environment variables
    echo "source /opt/intel/openvino/bin/setupvars.sh" >> /root/.bashrc &&\
    #
    # Cleaning
    cd ${WORK_DIR} &&\
    rm -rf * &&\
    /opt/miniconda/bin/conda clean -ya &&\
    rm -rf /opt/miniconda/pkgs &&\
    rm -rf /var/lib/apt/lists/*

#
# Set environment variables as in ${INTEL_OPENVINO_DIR}/bin/setupvars.sh
ENV INTEL_OPENVINO_DIR /opt/intel/openvino
ENV LD_LIBRARY_PATH ${INTEL_OPENVINO_DIR}/opencv/lib:${INTEL_OPENVINO_DIR}/deployment_tools/ngraph/lib:/opt/intel/opencl:${INTEL_OPENVINO_DIR}/deployment_tools/inference_engine/external/hddl/lib:${INTEL_OPENVINO_DIR}/deployment_tools/inference_engine/external/gna/lib:${INTEL_OPENVINO_DIR}/deployment_tools/inference_engine/external/mkltiny_lnx/lib:${INTEL_OPENVINO_DIR}/deployment_tools/inference_engine/external/tbb/lib:${INTEL_OPENVINO_DIR}/deployment_tools/inference_engine/lib/intel64
ENV INTEL_CVSDK_DIR ${INTEL_OPENVINO_DIR}
ENV OpenCV_DIR ${INTEL_OPENVINO_DIR}/opencv/cmake
ENV InferenceEngine_DIR ${INTEL_OPENVINO_DIR}/deployment_tools/inference_engine/share
ENV PYTHONPATH ${INTEL_OPENVINO_DIR}/python/python3.7:${INTEL_OPENVINO_DIR}/python/python3:${INTEL_OPENVINO_DIR}/deployment_tools/open_model_zoo/tools/accuracy_checker:${INTEL_OPENVINO_DIR}/deployment_tools/model_optimizer
ENV PATH ${INTEL_OPENVINO_DIR}/deployment_tools/model_optimizer:/opt/miniconda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:${PATH}
ENV HDDL_INSTALL_DIR ${INTEL_OPENVINO_DIR}/deployment_tools/inference_engine/external/hddl

#
# Exclude UDEV by rebuilding libusb without UDEV support
RUN cp ${INTEL_OPENVINO_DIR}/deployment_tools/inference_engine/external/97-myriad-usbboot.rules /etc/udev/rules.d/ &&\
    ldconfig &&\
    cd /opt && wget --quiet --no-check-certificate http://github.com/libusb/libusb/archive/v1.0.22.zip -O /opt/v1.0.22.zip &&\
    unzip v1.0.22.zip && cd libusb-1.0.22 &&\
    ./bootstrap.sh &&\
    ./configure --disable-udev --enable-shared &&\
    make -j4

RUN apt-get update &&\
    apt-get install -y --no-install-recommends libusb-1.0-0-dev &&\
    cd /opt &&\
    rm -rf /var/lib/apt/lists/* &&\
    cd /opt/libusb-1.0.22/libusb &&\
    /bin/mkdir -p '/usr/local/lib' &&\
    /bin/bash ../libtool --mode=install /usr/bin/install -c libusb-1.0.la '/usr/local/lib' &&\
    /bin/mkdir -p '/usr/local/include/libusb-1.0' &&\
    /usr/bin/install -c -m 644 libusb.h '/usr/local/include/libusb-1.0' &&\
    /bin/mkdir -p '/usr/local/lib/pkgconfig' &&\
    cd /opt/libusb-1.0.22/ &&\
    /usr/bin/install -c -m 644 libusb-1.0.pc '/usr/local/lib/pkgconfig' &&\
    ldconfig

#
# Install ML solution
RUN apt-get update && apt-get install -y --no-install-recommends \
    nginx \
    supervisor &&\
    pip install \
        numpy \
        azure-iot-device
        
ADD . ${WORK_DIR}
ADD etc /etc

RUN rm -rf /var/lib/apt/lists/* &&\
    rm /etc/nginx/sites-enabled/default &&\
    cp ${WORK_DIR}/nginx/app /etc/nginx/sites-available/ &&\
    ln -s /etc/nginx/sites-available/app /etc/nginx/sites-enabled/ &&\
    pip install -r ${WORK_DIR}/requirements.txt &&\
    /opt/miniconda/bin/conda clean -ya &&\
    rm -rf /opt/miniconda/pkgs &&\
    rm -rf /var/lib/apt/lists/*

EXPOSE 5001
CMD ["supervisord", "-c", "/code/etc/supervisord.conf"]

In [ ]:
# Update docker file with custom environment variable. IoT Edge device's connection string
filePath = isSolutionPath+"/Dockerfile"
file = open(filePath)
dockerFileTemplate = file.read()
dockerFileTemplate = dockerFileTemplate.replace("AIX_OPENVINO_TOOLKIT_DOWNLOAD_LINK", "\""+openVinoToolkitDownloadLink+"\"")

with open(filePath, 'wt', encoding='utf-8') as outputFile:
    outputFile.write(dockerFileTemplate)

### 5.4. Create Local Docker Image
We create the image locally, without hostring it yet in a container registry like docker.com or ACR or local registry


In [ ]:
%%bash -s "$containerImageName" "$isSolutionPath"
docker build -t $1 --file ./$2/Dockerfile ./$2

### 5.5. Host the local docker image in local docker registry
We will be hosting the docker image on a local registry which is either on the edge device, or on the development device which is on the same network as the edge device. We prefer this method only at the development stage as:
- Docker image size ~ 1-2 GB
- Hosting image on Azure Container Registry (ACR) means each time we compile the image, we will send it, push it to cloud and then pull all the way back to edge device. In case of low Internet connection bandwidth, this may be an issue or time consuming operation.
- There is an option where you can send smaller size DockerFile and dependencies to ACR and compile there. But in this case, 1) again you need to pull back the 1-2 GB size image to edge device 2) Even in case of a small code change in the last layers of the docker image, ACR will re-compile every layer from scratc which may take additional ~40 minutes...

So for above reasons, at the development and testing stages, we will use local container registry. Below is very simple steps to create your own docker registry by running the following bash commands manually:  

<span style="color:red; font-weight: bold; font-size:1.1em;"> [!Important] </span>  
In the below instructions we use **"mkregistry:55000"** as static text which is stored in the variable **localacrServiceName**. Value of this variable set in the first step/section of this sample. You can change this value to anything else but in case, you should update the commands in this static text with the new value.


1) Create a folder to host local docker registry configuration folder (i.e. we name it "mkregistry").
```shell
mkdir mkregistry
```
2) Inside the "mkregistry" directory, create a congiguration file named "docker-compose.yml" with the following content in:

```yml
version: '3.0'
 
services:
  mkregistry:
    image: registry:latest
    container_name: mkregistry
    volumes:
      - registry:/var/lib/registry
    ports:
      - "55000:5000"
    restart: unless-stopped
volumes:
  registry:
```

3) Install "docker-compose" tool. Run the following shell command to install it.  
```shell
sudo apt-get -y install docker-compose
```

4) Run the following command inside the "mkregistry" directory with "docker-compose.yml" file in it.  
```shell
sudo docker-compose up -d
```

<span style="color:red; font-weight: bold; font-size:1.1em;"> [!Important] </span>  
> When you restart the machine etc. you need to re-run above command to start the local regisrty service (you can search for methods to run it as auto run service)
when you type "docker container ls" shell command. You should see the following line to confirm that the service is running, you can validate with below next step.
```
f63fd9492a03        registry:latest                              "/entrypoint.sh /etc…"    20 hours ago        Up 19 hours         0.0.0.0:55000->5000/tcp                                                mkregistry
m
```

5) In your Internet explorer - web browser, you browse to following address to see the images in your local repository (initially it is empty...)  
```
http://localhost:55000/v2/_catalog
```

6) Edit "hosts" file to assign a name resolution setting for local docker registry. First open the "/etc/hosts" file with your favorite text editor:  
```shell
sudo nano /etc/hosts
```

than add the following line under "127.0.0.0 localhost"  

```shell
127.0.0.1	mkregistry
```

so the content of "/etc/hosts" file should look like something:  
```shell
127.0.0.1	localhost
127.0.0.1	mkregistry
127.0.1.1	mknuc01

# The following lines are desirable for IPv6 capable hosts
::1     ip6-localhost ip6-loopback
fe00::0 ip6-localnet
ff00::0 ip6-mcastprefix
ff02::1 ip6-allnodes
ff02::2 ip6-allrouters
```

thats all, now we have local docker registy with following addres: mkregistry:55000  

With following code, we will tag our local image and pull it into our local docker registry. So when we deploy our IotEdge manifest to IoT Hub, the edge devices will pull it from this registry. In production stage, we will host the image in ACR, more centric and managable store.

In [ ]:
%%bash -s "$containerImageName" "$localacrServiceName"
docker tag $1 $2/$1
docker push $2/$1 

<span style="color:red; font-weight: bold; font-size:1.1em;"> [!Important] </span>  
If you are in production, you can push the image into ACR (and not to local repository as in the above cell). Uncomment and use the following command to push it into ACR. Also dont forget to set the  useACRFlag  to TRUE in the next step/section of the sample where you deploy the image into edge devices. So the edge device will pull modules from the ACR and not from local registry...

<span style="color:red; font-weight: bold; font-size:1.1em;"> [!Warning] </span>  
Execution of following cell may take up to ~40 - 45 minutes to complete! It will be using Azure Constainer Registry (ACR) to compile a docker image.

In [ ]:
#!az acr build --image $containerImageName --registry $acrServiceName --file ./$isSolutionPath/Dockerfile ./$isSolutionPath